In [48]:
import lsqfit
import jax
import jax.numpy  as jnp
import gvar       as gv
import corrfitter as cf
import numpy      as np

In [8]:
from b2heavy.TwoPointFunctions.types2pts  import CorrelatorIO, Correlator
from b2heavy.TwoPointFunctions.fitter     import CorrFitter

In [19]:
ens = 'Coarse-1'
mes = 'Dst'
mom = '200'
binsize = 11
data_dir = '/Users/pietro/code/data_analysis/BtoD/Alex/'

io     = CorrelatorIO(ens,mes,mom,PathToDataDir=data_dir)
corr   = Correlator(io,jkBin=binsize)


Nexc   = 2
trange = (10,19) 
fitter = CorrFitter(corr,smearing=['1S-1S','d-d','d-1S'])

{'E': [1.161(52), -1.81(58), -1.09(80), -2.4(2.5)],
 'Z_1S_Bot': [0 ± 1.2, -1.2(1.2), 0.5(1.5), 0.5(1.5)],
 'Z_1S_Par': [0 ± 1.2, -1.2(1.2), 0.5(1.5), 0.5(1.5)],
 'Z_d-1S_Bot': [0.5(1.7), 0.5(1.7)],
 'Z_d-1S_Par': [0.5(1.7), 0.5(1.7)],
 'Z_d_Bot': [-1.93(51), -3.0(1.5), 0.5(1.5), 0.5(1.5)],
 'Z_d_Par': [-1.93(51), -5.5(2.0), 0.5(1.5), 0.5(1.5)]}

In [52]:
# single model for corrfitter
x,y = corr.format(smearing=['1S-1S'], polarization=['Par'])
data = {'1S-1S.Par': y['1S-1S','Par']}


p = fitter.set_priors_phys(Nstates=2)
pr = {
    'dE' : [p['E'][0], np.exp(p['E'][2])],
    'dEo': [p['E'][0] + np.exp(p['E'][0]), np.exp(p['E'][3])],
    'Z_1S_Par' : [np.exp(p['Z_1S_Par'][0]),np.exp(p['Z_1S_Par'][2])],
    'Z_1S_Paro': [np.exp(p['Z_1S_Par'][1]),np.exp(p['Z_1S_Par'][3])],
}


model = cf.Corr2(
    datatag = '1S-1S.Par',
    a       = (f'Z_1S_Par',f'Z_1S_Paro'),
    b       = (f'Z_1S_Par',f'Z_1S_Paro'),
    dE      = ('dE'       ,'dEo'      ),
    s       = (1,-1.),
    tdata   = x['1S-1S','Par'],
    tfit    = np.arange(min(trange),max(trange)+1),
    tp      = corr.Nt,
)

cfitter = cf.CorrFitter(models=model)

In [72]:
# Multiple model corrfitter
p = fitter.set_priors_phys(Nstates=2)
pr = {
    'dE' : [p['E'][0], np.exp(p['E'][2])],
    'dEo': [p['E'][0] + np.exp(p['E'][0]), np.exp(p['E'][3])],
    'Z_1S_Par'   : [np.exp(p['Z_1S_Par'][0])  ,p['Z_1S_Par'][2]],
    'Z_1S_Paro'  : [np.exp(p['Z_1S_Par'][1])  ,p['Z_1S_Par'][3]],
    'Z_d_Par'    : [np.exp(p['Z_d_Par'][0])   ,p['Z_d_Par'][2]],
    'Z_d_Paro'   : [np.exp(p['Z_d_Par'][1])   ,p['Z_d_Par'][3]],
    'Z_d-1S_Par' : [np.sqrt(np.exp(p['Z_d_Par']+p['Z_1S_Par'])),p['Z_d-1S_Par'][0]],
    'Z_d-1S_Paro': [np.sqrt(np.exp(p['Z_d_Par']+p['Z_1S_Par'])),p['Z_d-1S_Par'][1]],
    'Z_1S_Bot'   : [np.exp(p['Z_1S_Bot'][0])  ,p['Z_1S_Bot'][2]],
    'Z_1S_Boto'  : [np.exp(p['Z_1S_Bot'][1])  ,p['Z_1S_Bot'][3]],
    'Z_d_Bot'    : [np.exp(p['Z_d_Bot'][0])   ,p['Z_d_Bot'][2]],
    'Z_d_Boto'   : [np.exp(p['Z_d_Bot'][1])   ,p['Z_d_Bot'][3]],
    'Z_d-1S_Bot' : [np.sqrt(np.exp(p['Z_d_Bot']+p['Z_1S_Bot'])),p['Z_d-1S_Bot'][0]],
    'Z_d-1S_Boto': [np.sqrt(np.exp(p['Z_d_Bot']+p['Z_1S_Bot'])),p['Z_d-1S_Bot'][1]],
}

models = []
data   = {}
for (sm,pol) in fitter.keys:
    sm1,sm2 = sm.split('-')
    mix = sm1!=sm2

    x,y = corr.format(smearing=[sm], polarization=[pol])
    data[f'{sm}.{pol}'] = y[sm,pol]

    model = cf.Corr2(
        datatag = f'{sm}.{pol}',
        a       = (f'Z_{sm1}_{pol}',f'Z_{sm1}_{pol}o'),
        b       = (f'Z_{sm2}_{pol}',f'Z_{sm2}_{pol}o'),
        dE      = ('dE'       ,'dEo'      ),
        s       = (1,-1.),
        tdata   = x[sm,pol],
        tfit    = np.arange(min(trange),max(trange)+1),
        tp      = corr.Nt,
    )

    models.append(model)

cfitter = cf.CorrFitter(models=models)


In [73]:
fit = cfitter.lsqfit(data=data, prior=pr)

In [74]:
print(fit)

Least Square Fit:
  chi2/dof [dof] = 43 [60]    Q = 0    logGBF = -80.512

Parameters:
     Z_1S_Bot 0     0.89 (14)     [   1.0 (1.2) ]  
              1    -0.15 (76)     [   0.5 (1.5) ]  
           dE 0    1.098 (11)     [  1.161 (52) ]  *
              1    0.033 (17)     [   0.34 (27) ]  *
    Z_1S_Boto 0     0.30 (36)     [   0.30 (36) ]  
              1     0.5 (1.5)     [   0.5 (1.5) ]  
          dEo 0    4.088 (44)     [   4.35 (22) ]  *
              1     0.10 (24)     [   0.10 (24) ]  
     Z_1S_Par 0     0.35 (51)     [   1.0 (1.2) ]  
              1     1.13 (16)     [   0.5 (1.5) ]  
    Z_1S_Paro 0     0.30 (36)     [   0.30 (36) ]  
              1     0.5 (1.5)     [   0.5 (1.5) ]  
      Z_d_Bot 0    0.104 (25)     [  0.145 (73) ]  
              1   -0.033 (88)     [   0.5 (1.5) ]  
     Z_d_Boto 0    0.050 (75)     [  0.050 (75) ]  
              1     0.5 (1.5)     [   0.5 (1.5) ]  
      Z_d_Par 0    0.035 (62)     [  0.145 (73) ]  *
              1    0.134 